In [1]:
from bs4 import BeautifulSoup
import numpy
import requests
import unicodedata

In [2]:
def get_request(url):
    try:
        document = requests.get(url)
        return document.text
    except Exception as e:
        print("Exception during get request {} e".format(url))
        return None

In [3]:
html_document = get_request("https://www.giallozafferano.it/ricette-cat/page2/")
soup = BeautifulSoup(html_document, "html.parser")
cnt = soup.find_all(attrs={'class': 'gz-title'})
img_link = soup.find_all("img",attrs={'class': 'lazyload'})

print(img_link[0]['data-src'])

https://www.giallozafferano.it/images/242-24261/Muffin-con-gocce-di-cioccolato_360x300.jpg


In [4]:
def parse_recipes_pages(dictionary,page_number=1):
    html_document = get_request("https://www.giallozafferano.it/ricette-cat/page{}/".format(page_number))
    if html_document is None: 
        return None
    soup = BeautifulSoup(html_document, "html.parser")
    title_containers = soup.find_all(attrs={'class': 'gz-title'})
    
    for titles in title_containers:
        recipe_name = titles.a["title"]
        name = dict_insert_new_recipe(recipe_name,dictionary)
        dictionary[name]["Link"] = titles.a["href"]
    
    return 1    
def generate_name(recipe_name,dictionary):
    new_name = recipe_name
    i=0
    while(new_name in dictionary):
        new_name = recipe_name + str(i)
        i+=1
    return new_name
        
def dict_insert_new_recipe(recipe_name,dictionary):
    if recipe_name in dictionary:
        name = generate_name(recipe_name,dictionary)
        dictionary[name] = {}
        return name
    else:
        dictionary[recipe_name] = {}
        return recipe_name

In [5]:
my_dict = {}
parse_recipes_pages(my_dict,3)

In [16]:
def parse_recipe(url,dictionary):
    soup = BeautifulSoup()
    html_document = get_request(url)
    if html_document is None: 
        return None
    
    print("Parsing recipe {}\n".format(url))
    soup = BeautifulSoup(html_document, "html.parser")
    ingredients_cnt = soup.find_all(attrs={"class":"gz-ingredient"})
    
    dictionary["Ingredienti"] = {}
    for ingredient in ingredients_cnt:
        name = ingredient.a.text
        grm = ingredient.span.text
        grm = grm.replace("\n", "")
        grm = grm.replace("\t","")
        dictionary["Ingredienti"][name] = grm 
    steps_cnt = soup.find_all(attrs={"class":"gz-content-recipe-step"})
    steps = ""
    for step in steps_cnt:
        for match in step.findAll('span'):
            match.decompose()
        for match in step.findAll('p'):
            steps += match.text
        steps = steps.replace(" .",".")
        steps = steps.replace(" ,",",")
        steps = steps.replace(r"\\'",r"'")
        
    dictionary["Steps"] = unicodedata.normalize("NFKC", steps)

In [17]:
for recipe in my_dict:
    url = my_dict[recipe]["Link"] 
    parse_recipe(url,my_dict[recipe])
my_dict

Parsing recipe https://ricette.giallozafferano.it/Sacher-torte.html

Parsing recipe https://ricette.giallozafferano.it/Risotto-allo-Zafferano.html

Parsing recipe https://ricette.giallozafferano.it/Polpette-al-sugo.html

Parsing recipe https://ricette.giallozafferano.it/Cream-tart.html

Parsing recipe https://ricette.giallozafferano.it/Focaccia-fugassa-alla-genovese.html

Parsing recipe https://ricette.giallozafferano.it/Torta-caprese.html

Parsing recipe https://ricette.giallozafferano.it/Pasta-con-asparagi-e-pancetta.html

Parsing recipe https://ricette.giallozafferano.it/Pollo-al-limone.html

Parsing recipe https://ricette.giallozafferano.it/Focaccia-morbida.html

Parsing recipe https://ricette.giallozafferano.it/Muffin-al-cioccolato.html

Parsing recipe https://ricette.giallozafferano.it/Cookies.html

Parsing recipe https://ricette.giallozafferano.it/Hummus.html

Parsing recipe https://ricette.giallozafferano.it/Gateau-di-patate.html



KeyboardInterrupt: 

In [ ]:
i = 1
recipes_dict = {}
while(True):
    print("Page Number: {}".format(i))
    oldSize = len(recipes_dict)
    parse_recipes_pages(recipes_dict,page_number=i)
    if oldSize == len(recipes_dict):
        break
    i+=1

In [18]:
from multiprocessing import Pool
import multiprocessing 
pool_worker = multiprocessing.cpu_count() -2
threadPool = Pool(pool_worker)


results = []

Parsing recipe https://ricette.giallozafferano.it/Tiramisu.html

Parsing recipe https://ricette.giallozafferano.it/Crepes-dolci-e-salate-ricetta-base.html
Parsing recipe https://ricette.giallozafferano.it/Pancakes-allo-sciroppo-d-acero.html


Parsing recipe https://ricette.giallozafferano.it/Torta-tenerina.html

Parsing recipe https://ricette.giallozafferano.it/Impasto-per-pizza.html

Parsing recipe https://ricette.giallozafferano.it/New-York-Cheesecake.html
Parsing recipe https://ricette.giallozafferano.it/Brownies.html
Parsing recipe https://ricette.giallozafferano.it/Spaghetti-all-Amatriciana.html
Parsing recipe https://ricette.giallozafferano.it/Spaghetti-alla-Carbonara.html




Parsing recipe https://ricette.giallozafferano.it/Tortino-di-cioccolato-con-cuore-fondente.html

Parsing recipe https://ricette.giallozafferano.it/Banana-bread.html

Parsing recipe https://ricette.giallozafferano.it/Parmigiana-di-melanzane.html

Parsing recipe https://ricette.giallozafferano.it/Spaghetti-al


Parsing recipe https://ricette.giallozafferano.it/Sushi.html
Parsing recipe https://ricette.giallozafferano.it/Zuppa-di-cipolle-gratinata.html


Parsing recipe https://ricette.giallozafferano.it/Risotto-alla-zucca.html

Parsing recipe https://ricette.giallozafferano.it/Crema-di-limoncello.html

Parsing recipe https://ricette.giallozafferano.it/Pollo-al-curry.html

Parsing recipe https://ricette.giallozafferano.it/Ossibuchi-alla-milanese.html

Parsing recipe https://ricette.giallozafferano.it/Ravioli-ricotta-e-spinaci.html

Parsing recipe https://ricette.giallozafferano.it/Cheesecake-alla-Nutella.html

Parsing recipe https://ricette.giallozafferano.it/Mezze-maniche-al-tonno.html

Parsing recipe https://ricette.giallozafferano.it/Orata-al-forno.html

Parsing recipe https://ricette.giallozafferano.it/Torta-di-banane.html

Parsing recipe https://ricette.giallozafferano.it/Frittata-di-spaghetti.html

Parsing recipe https://ricette.giallozafferano.it/Crostata-morbida-al-cioccolato.html

Par


Parsing recipe https://ricette.giallozafferano.it/Pizza-in-teglia-alta-e-morbida.html

Parsing recipe https://ricette.giallozafferano.it/Couscous-freddo-di-pollo.html

Parsing recipe https://ricette.giallozafferano.it/Carne-alla-pizzaiola.html

Parsing recipe https://ricette.giallozafferano.it/Pasta-biscotto.html

Parsing recipe https://ricette.giallozafferano.it/Croissant.html

Parsing recipe https://ricette.giallozafferano.it/Torta-salata-di-zucchine.html

Parsing recipe https://ricette.giallozafferano.it/Orecchiette-con-le-cime-di-rapa.html

Parsing recipe https://ricette.giallozafferano.it/Pasta-al-salmone.html

Parsing recipe https://ricette.giallozafferano.it/Polpettone-al-forno.html

Parsing recipe https://ricette.giallozafferano.it/Torta-sbriciolata-alla-Nutella.html

Parsing recipe https://ricette.giallozafferano.it/Pizzette-rosse.html

Parsing recipe https://ricette.giallozafferano.it/Ravioli-di-carne-giapponesi-Gyoza.html

Parsing recipe https://ricette.giallozafferano.it/P


Parsing recipe https://ricette.giallozafferano.it/Graffe.html

Parsing recipe https://ricette.giallozafferano.it/Focaccia-con-pomodorini-e-origano.html

Parsing recipe https://ricette.giallozafferano.it/rosti.html
Parsing recipe https://ricette.giallozafferano.it/Crepes-alla-crema-di-asparagi.html


Parsing recipe https://ricette.giallozafferano.it/Torta-cremosa-di-mele.html

Parsing recipe https://ricette.giallozafferano.it/Tonno-in-crosta-di-pistacchi.html

Parsing recipe https://ricette.giallozafferano.it/Zeppole-di-San-Giuseppe.html

Parsing recipe https://ricette.giallozafferano.it/Fajitas-di-carne-e-pollo.html

Parsing recipe https://ricette.giallozafferano.it/Plumcake-salato.html

Parsing recipe https://ricette.giallozafferano.it/Torta-moderna.html

Parsing recipe https://ricette.giallozafferano.it/Frittata-di-patate.html

Parsing recipe https://ricette.giallozafferano.it/Risotto-ai-carciofi.html

Parsing recipe https://ricette.giallozafferano.it/Croque-monsieur.html

Parsing r


Parsing recipe https://ricette.giallozafferano.it/ClubHouse-Sandwich.html

Parsing recipe https://ricette.giallozafferano.it/Pasta-al-forno-bianca.html

Parsing recipe https://ricette.giallozafferano.it/Naan.html

Parsing recipe https://ricette.giallozafferano.it/Pappardelle-con-speck-e-funghi-porcini.html

Parsing recipe https://ricette.giallozafferano.it/Vellutata-di-cavolfiore.html

Parsing recipe https://ricette.giallozafferano.it/Patate-al-forno-ripiene.html

Parsing recipe https://ricette.giallozafferano.it/Crescentine-o-tigelle.html

Parsing recipe https://ricette.giallozafferano.it/Torta-al-cioccolato-senza-farina.html

Parsing recipe https://ricette.giallozafferano.it/Mug-cake.html

Parsing recipe https://ricette.giallozafferano.it/Feta-al-forno.html

Parsing recipe https://ricette.giallozafferano.it/Asparagi-alla-milanese.html

Parsing recipe https://ricette.giallozafferano.it/Tzatziki-smirneico.html

Parsing recipe https://ricette.giallozafferano.it/Strudel-con-ricotta-e-sp

...


Parsing recipe https://ricette.giallozafferano.it/Panzerotti-con-ricotta-cioccolato-e-granella-di-nocciole.html

Parsing recipe https://ricette.giallozafferano.it/Piadina-con-fiori-di-zucca-ripieni-e-fritti.html

Parsing recipe https://ricette.giallozafferano.it/Panettone-con-fichi-e-mandorle.html

Parsing recipe https://ricette.giallozafferano.it/Caffe-meringato.html

Parsing recipe https://ricette.giallozafferano.it/Pici-toscani.html

Parsing recipe https://ricette.giallozafferano.it/Nidi-capresi-di-spaghetti.html

Parsing recipe https://ricette.giallozafferano.it/Mini-strudel-ai-frutti-di-bosco-e-ricotta.html

Parsing recipe https://ricette.giallozafferano.it/Pasta-di-legumi-con-pesto-di-verza-e-nocciole.html

Parsing recipe https://ricette.giallozafferano.it/Strudel-di-mele-cipolle-e-speck.html

Parsing recipe https://ricette.giallozafferano.it/Cannoncini-salati-con-caprino-e-pancetta.html

Parsing recipe https://ricette.giallozafferano.it/Tempura-di-verdure-autunnali-e-carne.html


Parsing recipe https://ricette.giallozafferano.it/Cous-cous-con-anguria-feta-e-pomodorini.html

Parsing recipe https://ricette.giallozafferano.it/Focaccia-di-grano-arso-con-patate-e-acciughe.html

Parsing recipe https://ricette.giallozafferano.it/Sfoglie-classiche-con-spinaci-uvetta-e-pinoli.html

Parsing recipe https://ricette.giallozafferano.it/Fichi-con-pecorino.html

Parsing recipe https://ricette.giallozafferano.it/Chips-di-patate-dolci-con-fonduta-al-Grana-Padano.html

Parsing recipe https://ricette.giallozafferano.it/Barchette-di-riso-ai-frutti-rossi.html

Parsing recipe https://ricette.giallozafferano.it/Bicchierino-lamponi-yogurt-e-Nutella.html

Parsing recipe https://ricette.giallozafferano.it/Schiacciata-alle-nespole-e-rosmarino.html

Parsing recipe https://ricette.giallozafferano.it/Castagne-al-sale.html

Parsing recipe https://ricette.giallozafferano.it/Sfoglie-alle-olive-con-pomodoro-e-briciole-di-feta.html

Parsing recipe https://ricette.giallozafferano.it/Cupcake-di-zu


Parsing recipe https://ricette.giallozafferano.it/Uovo-di-Pasqua-gourmet.html

Parsing recipe https://ricette.giallozafferano.it/Chips-di-tuberi-e-maionese-allo-zenzero.html
Parsing recipe https://ricette.giallozafferano.it/Sfogliatine-di-pere-con-Nutella.html


Parsing recipe https://ricette.giallozafferano.it/Fiori-di-Eglefino-al-limone-e-paprika-con-patate.html

Parsing recipe https://ricette.giallozafferano.it/Teglia-di-patate-alla-paprika.html

Parsing recipe https://ricette.giallozafferano.it/Tartellette-all-anguria.html

Parsing recipe https://ricette.giallozafferano.it/Halloween-cake-con-Pavesini.html

Parsing recipe https://ricette.giallozafferano.it/Galletto-al-sale.html

Parsing recipe https://ricette.giallozafferano.it/Schiacciate-con-cime-di-rapa-piccanti-e-salsiccia.html

Parsing recipe https://ricette.giallozafferano.it/Pere-e-speck-con-frutta-secca.html

Parsing recipe https://ricette.giallozafferano.it/Calamari-ripieni-di-bieta-uvetta-e-mandorle.html

Parsing recipe h


Parsing recipe https://ricette.giallozafferano.it/Galletto-fritto.html

Parsing recipe https://ricette.giallozafferano.it/Tortini-alle-tre-carote.html

Parsing recipe https://ricette.giallozafferano.it/Polpettine-mostruose.html

Parsing recipe https://ricette.giallozafferano.it/Tronco-delle-feste-con-Nutella.html

Parsing recipe https://ricette.giallozafferano.it/Pizza-integrale-con-radicchio-e-cipolle.html

Parsing recipe https://ricette.giallozafferano.it/Cornbread-burger-con-porcino.html

Parsing recipe https://ricette.giallozafferano.it/Pasta-Babbo-Natale.html

Parsing recipe https://ricette.giallozafferano.it/Sfoglia-classica-con-scampi-saltati-al-balsamico.html

Parsing recipe https://ricette.giallozafferano.it/Granita-bicolore-con-finto-gelato.html

Parsing recipe https://ricette.giallozafferano.it/Bruschette-di-panettone-al-gorgonzola-e-ribes.html

Parsing recipe https://ricette.giallozafferano.it/Tortino-pasqualino.html

Parsing recipe https://ricette.giallozafferano.it/Bicch


Parsing recipe https://ricette.giallozafferano.it/Ciambelline-al-cioccolato-e-nocciole.html

Parsing recipe https://ricette.giallozafferano.it/Crostatine-con-crema-di-Skyr-pere-e-anacardi.html

Parsing recipe https://ricette.giallozafferano.it/Cupcake-salati-con-frosting-di-patate.html

Parsing recipe https://ricette.giallozafferano.it/Cestini-di-foglie-di-vite-con-porcini-e-Grana-Padano.html

Parsing recipe https://ricette.giallozafferano.it/Torcelli-con-crema-di-lenticchie-e-guanciale.html

Parsing recipe https://ricette.giallozafferano.it/Azevias.html

Parsing recipe https://ricette.giallozafferano.it/Schiacciate-con-insalata-di-patate-all-Asiago.html

Parsing recipe https://ricette.giallozafferano.it/Fiori-di-Eglefino-zucca-e-pesto.html

Parsing recipe https://ricette.giallozafferano.it/Crostone-con-zucca-grigliata-e-robiola.html

Parsing recipe https://ricette.giallozafferano.it/Cialda-di-pandoro-con-mousse-di-ricotta-e-Nutella.html

Parsing recipe https://ricette.giallozafferano

In [21]:
for recipe in recipes_dict:
    url = recipes_dict[recipe]["Link"] 
    parse_recipe(url,recipes_dict[recipe])

Parsing recipe https://ricette.giallozafferano.it/Tiramisu.html

Parsing recipe https://ricette.giallozafferano.it/Pancakes-allo-sciroppo-d-acero.html

Parsing recipe https://ricette.giallozafferano.it/Spaghetti-alla-Carbonara.html

Parsing recipe https://ricette.giallozafferano.it/Crepes-dolci-e-salate-ricetta-base.html

Parsing recipe https://ricette.giallozafferano.it/Impasto-per-pizza.html

Parsing recipe https://ricette.giallozafferano.it/New-York-Cheesecake.html

Parsing recipe https://ricette.giallozafferano.it/Spaghetti-all-Amatriciana.html

Parsing recipe https://ricette.giallozafferano.it/Torta-tenerina.html

Parsing recipe https://ricette.giallozafferano.it/Tortino-di-cioccolato-con-cuore-fondente.html

Parsing recipe https://ricette.giallozafferano.it/Brownies.html

Parsing recipe https://ricette.giallozafferano.it/Banana-bread.html

Parsing recipe https://ricette.giallozafferano.it/Besciamella.html

Parsing recipe https://ricette.giallozafferano.it/Spaghetti-alle-vongole.h

Parsing recipe https://ricette.giallozafferano.it/Zuppa-di-cipolle-gratinata.html

Parsing recipe https://ricette.giallozafferano.it/Sushi.html

Parsing recipe https://ricette.giallozafferano.it/Crema-di-limoncello.html

Parsing recipe https://ricette.giallozafferano.it/Risotto-alla-zucca.html

Parsing recipe https://ricette.giallozafferano.it/Pollo-al-curry.html

Parsing recipe https://ricette.giallozafferano.it/Ravioli-ricotta-e-spinaci.html

Parsing recipe https://ricette.giallozafferano.it/Ossibuchi-alla-milanese.html

Parsing recipe https://ricette.giallozafferano.it/Mezze-maniche-al-tonno.html

Parsing recipe https://ricette.giallozafferano.it/Orata-al-forno.html

Parsing recipe https://ricette.giallozafferano.it/Cheesecake-alla-Nutella.html

Parsing recipe https://ricette.giallozafferano.it/Frittata-di-spaghetti.html

Parsing recipe https://ricette.giallozafferano.it/Torta-di-banane.html

Parsing recipe https://ricette.giallozafferano.it/Crostata-morbida-al-cioccolato.html

Pars

Parsing recipe https://ricette.giallozafferano.it/Polpette-di-zucchine.html

Parsing recipe https://ricette.giallozafferano.it/Pizza-in-teglia-alta-e-morbida.html

Parsing recipe https://ricette.giallozafferano.it/Couscous-freddo-di-pollo.html

Parsing recipe https://ricette.giallozafferano.it/Carne-alla-pizzaiola.html

Parsing recipe https://ricette.giallozafferano.it/Pasta-biscotto.html

Parsing recipe https://ricette.giallozafferano.it/Croissant.html

Parsing recipe https://ricette.giallozafferano.it/Torta-salata-di-zucchine.html

Parsing recipe https://ricette.giallozafferano.it/Pasta-al-salmone.html

Parsing recipe https://ricette.giallozafferano.it/Polpettone-al-forno.html

Parsing recipe https://ricette.giallozafferano.it/Orecchiette-con-le-cime-di-rapa.html

Parsing recipe https://ricette.giallozafferano.it/Torta-sbriciolata-alla-Nutella.html

Parsing recipe https://ricette.giallozafferano.it/Pizzette-rosse.html

Parsing recipe https://ricette.giallozafferano.it/Ravioli-di-carn

Parsing recipe https://ricette.giallozafferano.it/Zuppa-di-lenticchie.html

Parsing recipe https://ricette.giallozafferano.it/Graffe.html

Parsing recipe https://ricette.giallozafferano.it/Focaccia-con-pomodorini-e-origano.html

Parsing recipe https://ricette.giallozafferano.it/Crepes-alla-crema-di-asparagi.html

Parsing recipe https://ricette.giallozafferano.it/rosti.html

Parsing recipe https://ricette.giallozafferano.it/Torta-cremosa-di-mele.html

Parsing recipe https://ricette.giallozafferano.it/Zeppole-di-San-Giuseppe.html

Parsing recipe https://ricette.giallozafferano.it/Tonno-in-crosta-di-pistacchi.html

Parsing recipe https://ricette.giallozafferano.it/Plumcake-salato.html

Parsing recipe https://ricette.giallozafferano.it/Fajitas-di-carne-e-pollo.html

Parsing recipe https://ricette.giallozafferano.it/Frittata-di-patate.html

Parsing recipe https://ricette.giallozafferano.it/Torta-moderna.html

Parsing recipe https://ricette.giallozafferano.it/Risotto-ai-carciofi.html

Parsin

Parsing recipe https://ricette.giallozafferano.it/ClubHouse-Sandwich.html

Parsing recipe https://ricette.giallozafferano.it/Naan.html

Parsing recipe https://ricette.giallozafferano.it/Quiche-Lorraine.html

Parsing recipe https://ricette.giallozafferano.it/Pasta-al-forno-bianca.html

Parsing recipe https://ricette.giallozafferano.it/Pappardelle-con-speck-e-funghi-porcini.html

Parsing recipe https://ricette.giallozafferano.it/Vellutata-di-cavolfiore.html

Parsing recipe https://ricette.giallozafferano.it/Patate-al-forno-ripiene.html

Parsing recipe https://ricette.giallozafferano.it/Crescentine-o-tigelle.html

Parsing recipe https://ricette.giallozafferano.it/Torta-al-cioccolato-senza-farina.html

Parsing recipe https://ricette.giallozafferano.it/Mug-cake.html

Parsing recipe https://ricette.giallozafferano.it/Feta-al-forno.html

Parsing recipe https://ricette.giallozafferano.it/Tzatziki-smirneico.html

Parsing recipe https://ricette.giallozafferano.it/Asparagi-alla-milanese.html

Par

...

Parsing recipe https://ricette.giallozafferano.it/Tagliatelle-ripiene-al-cotechino.html

Parsing recipe https://ricette.giallozafferano.it/Caffe-meringato.html

Parsing recipe https://ricette.giallozafferano.it/Piadina-con-fiori-di-zucca-ripieni-e-fritti.html

Parsing recipe https://ricette.giallozafferano.it/Nidi-capresi-di-spaghetti.html

Parsing recipe https://ricette.giallozafferano.it/Panettone-con-fichi-e-mandorle.html

Parsing recipe https://ricette.giallozafferano.it/Pasta-di-legumi-con-pesto-di-verza-e-nocciole.html

Parsing recipe https://ricette.giallozafferano.it/Pici-toscani.html

Parsing recipe https://ricette.giallozafferano.it/Cannoncini-salati-con-caprino-e-pancetta.html

Parsing recipe https://ricette.giallozafferano.it/Lasagne-di-pancarre-con-formaggio-e-ragu-di-verdure.html

Parsing recipe https://ricette.giallozafferano.it/Tempura-di-verdure-autunnali-e-carne.html

Parsing recipe https://ricette.giallozafferano.it/Anatra-alla-melagrana.html

Parsing recipe https://

Parsing recipe https://ricette.giallozafferano.it/Frittelle-di-castagne-sfiziose.html

Parsing recipe https://ricette.giallozafferano.it/Sfoglie-classiche-con-spinaci-uvetta-e-pinoli.html

Parsing recipe https://ricette.giallozafferano.it/Chips-di-patate-dolci-con-fonduta-al-Grana-Padano.html

Parsing recipe https://ricette.giallozafferano.it/Barchette-di-riso-ai-frutti-rossi.html

Parsing recipe https://ricette.giallozafferano.it/Torta-di-panettone-gastronomico.html

Parsing recipe https://ricette.giallozafferano.it/Castagne-al-sale.html

Parsing recipe https://ricette.giallozafferano.it/Cupcake-di-zucca-e-carote.html

Parsing recipe https://ricette.giallozafferano.it/Schiacciata-alle-nespole-e-rosmarino.html

Parsing recipe https://ricette.giallozafferano.it/Sfoglie-alle-olive-con-pomodoro-e-briciole-di-feta.html

Parsing recipe https://ricette.giallozafferano.it/Piadina-con-nduja-salsiccia-e-pomodorini.html

Parsing recipe https://ricette.giallozafferano.it/Risotto-con-pesto-di-spin

Parsing recipe https://ricette.giallozafferano.it/Pere-e-speck-con-frutta-secca.html

Parsing recipe https://ricette.giallozafferano.it/Tartellette-all-anguria.html

Parsing recipe https://ricette.giallozafferano.it/Tacos-con-mango-papaya-e-Nutella.html

Parsing recipe https://ricette.giallozafferano.it/Schiacciate-con-cime-di-rapa-piccanti-e-salsiccia.html

Parsing recipe https://ricette.giallozafferano.it/Calamari-ripieni-di-bieta-uvetta-e-mandorle.html

Parsing recipe https://ricette.giallozafferano.it/Biscotti-di-zucca-della-strega.html

Parsing recipe https://ricette.giallozafferano.it/Galletto-al-sale.html

Parsing recipe https://ricette.giallozafferano.it/Halloween-cake-con-Pavesini.html

Parsing recipe https://ricette.giallozafferano.it/Barchette-di-patate-e-formaggio-cremoso.html

Parsing recipe https://ricette.giallozafferano.it/Fiori-di-Eglefino-al-limone-e-paprika-con-patate.html

Parsing recipe https://ricette.giallozafferano.it/Teglia-di-patate-alla-paprika.html

Parsing 

Parsing recipe https://ricette.giallozafferano.it/Cornbread-burger-con-porcino.html

Parsing recipe https://ricette.giallozafferano.it/Pizza-integrale-con-radicchio-e-cipolle.html

Parsing recipe https://ricette.giallozafferano.it/Pasta-Babbo-Natale.html

Parsing recipe https://ricette.giallozafferano.it/Tortini-alle-tre-carote.html

Parsing recipe https://ricette.giallozafferano.it/Sfoglia-classica-con-scampi-saltati-al-balsamico.html

Parsing recipe https://ricette.giallozafferano.it/Tortini-di-pandoro-con-mele-noci-e-Nutella.html

Parsing recipe https://ricette.giallozafferano.it/Piadina-con-crema-al-cioccolato-e-frutta.html

Parsing recipe https://ricette.giallozafferano.it/Tronco-delle-feste-con-Nutella.html

Parsing recipe https://ricette.giallozafferano.it/Piadina-con-gamberi-fiori-di-zucca-e-insalata.html

Parsing recipe https://ricette.giallozafferano.it/Granita-bicolore-con-finto-gelato.html

Parsing recipe https://ricette.giallozafferano.it/Ostriche-gratinate-ai-funghi.html


Parsing recipe https://ricette.giallozafferano.it/Crostatine-con-crema-di-Skyr-pere-e-anacardi.html

Parsing recipe https://ricette.giallozafferano.it/Cupcake-salati-con-frosting-di-patate.html

Parsing recipe https://ricette.giallozafferano.it/Fiori-di-Eglefino-zucca-e-pesto.html

Parsing recipe https://ricette.giallozafferano.it/Azevias.html

Parsing recipe https://ricette.giallozafferano.it/Cestini-di-foglie-di-vite-con-porcini-e-Grana-Padano.html

Parsing recipe https://ricette.giallozafferano.it/Crostone-con-zucca-grigliata-e-robiola.html

Parsing recipe https://ricette.giallozafferano.it/Schiacciate-con-insalata-di-patate-all-Asiago.html

Parsing recipe https://ricette.giallozafferano.it/Pavesini-renna.html

Parsing recipe https://ricette.giallozafferano.it/Cialda-di-pandoro-con-mousse-di-ricotta-e-Nutella.html

Parsing recipe https://ricette.giallozafferano.it/Pane-al-sidro.html

Parsing recipe https://ricette.giallozafferano.it/Torcelli-con-crema-di-lenticchie-e-guanciale.html


In [23]:
recipes_dict

{'Tiramisù': {'Link': 'https://ricette.giallozafferano.it/Tiramisu.html',
  'Ingredienti': {'Mascarpone': '750g',
   'Uova': '(freschissime, circa 5 medie)260g',
   'Savoiardi': '250g',
   'Zucchero': '120g',
   'Caffè': '(della moka,  zuccherato a piacere)300g',
   'Cacao amaro in polvere': 'q.b.'},
  'Steps': "Per preparare il tiramisù preparate il caffé con la moka per ottenerne 300 g, poi zuccherate a piacere (noi abbiamo messo un cucchiaino) e lasciatelo raffreddare in una ciotolina bassa e ampia. Separate le uova dividendo gli albumi dai tuorli, ricordando che per montare bene gli albumi non dovranno presentare alcuna traccia di tuorlo. Montate i tuorli con le fruste elettriche, versando solo metà dose di zucchero. Non appena il composto sarà diventato chiaro e spumoso, e con le fruste ancora in funzione, potrete aggiungere il mascarpone, poco alla volta. Incorporato tutto il formaggio avrete ottenuto una crema densa e compatta ; tenetela da parte. Pulite molto bene le fruste e p

In [24]:
import json
with open("recipes.json", "w") as outfile:
    json.dump(recipes_dict, outfile)